# Example of Neural Network Model
This is an example of a simple implementation of a neural network model. Similar, but not equal to 

> Bengio, Y., Ducharme, R., Vincent, P., & Jauvin, C. (2003). A neural probabilistic language model. *Journal of machine learning research*, *3*(Feb), 1137-1155

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook

In [2]:
from langmodels.corpora.moviedialog import MovieDialogCollection

In [3]:
db_name = 'movie-dialogs'
collection = 'lines'
m = {'$match': {'character.movie.id': 'm42'}}
p = {'$project': {'_id': 0, 'id': 1, 'text': 1}}
pipeline = [m, p]

In [4]:
m42 = MovieDialogCollection(db_name, collection, 
                              use_pos=False, pipeline=pipeline)

In [5]:
V = m42.vocabulary

## Indexing words and create sequences

In [6]:
idx2w = {i:w for (i, w) in enumerate(V)}
w2idx = {w:i for (i, w) in enumerate(V)}

In [7]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [8]:
sequences = []
for doc, tokens in m42.get_tokens():
    sequences.append([w2idx[t] for t in tokens])
sequences = np.array(pad_sequences(sequences, padding='pre'))

## Create training and target labels

In [9]:
from keras.utils import to_categorical

In [10]:
X, y = sequences[:,:-1], sequences[:,-1]

In [11]:
y = to_categorical(y, num_classes=len(V))

In [12]:
print(X.shape, y.shape)

(834, 106) (834, 1363)


## Define the embedding layer
The embedding layer is based on the size of the vocabulary and the length of input sequences, plus the number of dimensions for vectors.

In [13]:
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

In [14]:
embedding_size = 50
E = Embedding(len(V), embedding_size, input_length=X.shape[1])

## Network architecture

In [15]:
model = Sequential()
model.add(E)
model.add(LSTM(100, return_sequences=False))
model.add(Dense(100, activation='relu'))
model.add(Dense(len(V), activation='softmax'))

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 106, 50)           68150     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 1363)              137663    
Total params: 276,313
Trainable params: 276,313
Non-trainable params: 0
_________________________________________________________________


## Train

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', 
              metrics=['accuracy'])
model.fit(X, y, batch_size=128, epochs=10)

/Users/alfio/Applications/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
834/834 [==============================] - 2s 3ms/step - loss: 5.3697 - accuracy: 0.0360
Epoch 2/10
834/834 [==============================] - 2s 2ms/step - loss: 5.2990 - accuracy: 0.0408
Epoch 3/10
834/834 [==============================] - 2s 2ms/step - loss: 5.2521 - accuracy: 0.0408
Epoch 4/10
834/834 [==============================] - 2s 2ms/step - loss: 5.2195 - accuracy: 0.0408
Epoch 5/10
834/834 [==============================] - 2s 2ms/step - loss: 5.1796 - accuracy: 0.0408
Epoch 6/10
834/834 [==============================] - 2s 2ms/step - loss: 5.1461 - accuracy: 0.0408
Epoch 7/10
834/834 [==============================] - 2s 2ms/step - loss: 5.1057 - accuracy: 0.0420
Epoch 8/10
834/834 [==============================] - 2s 2ms/step - loss: 5.0625 - accuracy: 0.0432
Epoch 9/10
834/834 [==============================] - 2s 2ms/step - loss: 5.0152 - accuracy: 0.0444
Epoch 10/10
834/834 [==============================] - 2s 2ms/step - loss: 4.9591 - accuracy: 0.0468

## Predict

In [19]:
test = sequences[42]
print([idx2w[x] for x in test if x != 0])
ask = np.array([0] + test[:-1])
print([idx2w[x] for x in ask if x != 0])

['we', 'read', 'five', 'times', 'that', 'you', 'were', 'killed', 'in', 'five', 'different', 'places']
['we', 'read', 'five', 'times', 'that', 'you', 'were', 'killed', 'in', 'five', 'different']


In [20]:
w = model.predict_classes(ask.reshape(1, -1))
print(w)

[1084]


In [21]:
model.predict(ask.reshape(1, -1))

array([[2.2945330e-06, 2.6858165e-06, 5.7038064e-06, ..., 2.7925596e-06,
        2.0020355e-03, 2.4844599e-06]], dtype=float32)

In [22]:
print(idx2w[w[0]])

time


## Assess the word embeddings

In [23]:
E.embeddings.shape

TensorShape([1363, 50])